In [ ]:
# Step1: Import the required libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.decomposition import PCA

In [2]:
# Step2: Read the data from train.csv & test.csv
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [3]:
df_train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
y_train=df_train['y'].values

In [5]:
cols=[c for c in df_train.columns if'X' in c]
print('Number of features:{}'.format(len(cols)))
print('Feature types:')
df_train[cols].dtypes.value_counts()

Number of features:376
Feature types:


int64     368
object      8
dtype: int64

In [6]:
counts = [[], [], []]
for c in cols:
    typ = df_train[c].dtype
    uniq = len(np.unique(df_train[c]))
    if uniq == 1:
        counts[0].append(c)
    elif uniq == 2 and typ == np.int64:
        counts[1].append(c)
    else:
        counts[2].append(c)
                

In [7]:
print('Constant features: {} Binary features: {} Categorical features: {}\n' .format(*[len(c) for c in counts]))

Constant features: 12 Binary features: 356 Categorical features: 8



In [8]:
print('Constant features:', counts[0])

Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']


In [9]:
print('Categorical features:', counts[2])

Categorical features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']


In [10]:
df_train.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df_test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


## remove columns ID and Y from the data as they are not used for learning

In [12]:
usable_columns = list(set(df_train.columns) - set(['ID', 'y']))

In [13]:
y_train = df_train['y'].values

In [14]:
id_test = df_test['ID'].values

In [15]:
x_train = df_train[usable_columns]
x_test = df_test[usable_columns]

In [16]:

def check_missing_values(df):
    if df.isnull().any().any():
        print("There are missing values in the dataframe")
    else:
        print("There are no missing values in the dataframe")
check_missing_values(x_train)
check_missing_values(x_test)

There are no missing values in the dataframe
There are no missing values in the dataframe



# Step3: If for any column(s), the variance is equal to zero, 
# then you need to remove those variable(s).
# Apply label encoder

In [17]:
for column in usable_columns:
    cardinality = len(np.unique(x_train[column]))
    if cardinality == 1:
        x_train.drop(column, axis=1) # Column with only one 
        # value is useless so we drop it
        x_test.drop(column, axis=1)
    if cardinality > 2: # Column is categorical
        mapper = lambda x: sum([ord(digit) for digit in x])
        x_train[column] = x_train[column].apply(mapper)
        x_test[column] = x_test[column].apply(mapper)
x_train.head()

,X64,X217,X258,X300,X2,X152,X242,X132,X43,X310,...,X156,X277,X349,X19,X308,X319,X87,X385,X53,X182
0,0,0,0,0,213,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,215,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,110,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,110,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,110,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


# Step4: Make sure the data is now changed into numerical

In [18]:
print('Feature types:')
x_train[cols].dtypes.value_counts()

Feature types:


int64    376
dtype: int64

# Step5: Perform dimensionality reduction
# Linear dimensionality reduction using Singular Value Decomposition of 
# the data to project it to a lower dimensional space.

In [19]:
from sklearn.decomposition import PCA
n_comp = 12
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(x_train)
pca2_results_test = pca.transform(x_test)


# Step6: Training using xgboost

In [20]:

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [21]:
x_train, x_valid, y_train, y_valid = train_test_split(
        pca2_results_train, 
        y_train, test_size=0.2, 
        random_state=4242)

In [22]:
import xgboost as xgb
d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
#d_test = xgb.DMatrix(x_test)
d_test = xgb.DMatrix(pca2_results_test)

In [23]:
params = {}
params['objective'] = 'reg:linear'
params['eta'] = 0.02
params['max_depth'] = 4

In [24]:
def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

In [25]:
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [26]:

clf = xgb.train(params, d_train, 
                1000, watchlist, early_stopping_rounds=50, 
                feval=xgb_r2_score, maximize=True, verbose_eval=10)

[19:27:56] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-030221e36e1a46bfb-1/xgboost/xgboost-ci-windows/src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:99.14834	train-r2:-58.35295	valid-rmse:98.26297	valid-r2:-67.63754
[10]	train-rmse:81.27653	train-r2:-38.88428	valid-rmse:80.36433	valid-r2:-44.91014
[20]	train-rmse:66.71610	train-r2:-25.87403	valid-rmse:65.77334	valid-r2:-29.75260
[30]	train-rmse:54.86913	train-r2:-17.17722	valid-rmse:53.89147	valid-r2:-19.64534
[40]	train-rmse:45.24710	train-r2:-11.36098	valid-rmse:44.22334	valid-r2:-12.90225
[50]	train-rmse:37.44856	train-r2:-7.46723	valid-rmse:36.37638	valid-r2:-8.40634
[60]	train-rmse:31.14585	train-r2:-4.85695	valid-rmse:30.02279	valid-r2:-5.40743
[70]	train-rmse:26.08417	train-r2:-3.10795	valid-rmse:24.91516	valid-r2:-3.41275
[80]	train-rmse:22.04312	train-r2:-1.93371	valid-rmse:20.83074	valid-r2:-2.08455
[90]	train-rmse:18.84671	train-

# Step7: Predict your test_df values using xgboost


In [27]:
p_test = clf.predict(d_test)

In [28]:
sub = pd.DataFrame()
sub['ID'] = id_test
sub['y'] = p_test
sub.to_csv('xgb.csv', index=False)

In [29]:

sub.head()

,ID,y
0,1,83.127792
1,2,96.628967
2,3,82.783844
3,4,76.797844
4,5,112.670700
